In [ ]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [ ]:
!pip install bert-serving-server

In [ ]:
!wget https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip; unzip uncased_L-12_H-768_A-12.zip

--2021-08-09 02:11:30--  https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.141.128, 142.251.2.128, 74.125.137.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.141.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 407727028 (389M) [application/zip]
Saving to: ‘uncased_L-12_H-768_A-12.zip’

uncased_L-12_H-768_ 100%[===================>] 388.84M   296MB/s    in 1.3s    

2021-08-09 02:11:32 (296 MB/s) - ‘uncased_L-12_H-768_A-12.zip’ saved [407727028/407727028]

Archive:  uncased_L-12_H-768_A-12.zip
   creating: uncased_L-12_H-768_A-12/
  inflating: uncased_L-12_H-768_A-12/bert_model.ckpt.meta  
  inflating: uncased_L-12_H-768_A-12/bert_model.ckpt.data-00000-of-00001  
  inflating: uncased_L-12_H-768_A-12/vocab.txt  
  inflating: uncased_L-12_H-768_A-12/bert_model.ckpt.index  
  inflating: uncased_L-12_H-768_A-12/bert_config.json  


In [ ]:
!nohup bert-serving-start -max_seq_len=128 -model_dir=uncased_L-12_H-768_A-12 > out.file 2>&1 &

In [ ]:
!pip install bert-serving-client

In [ ]:
from bert_serving.client import BertClient
bc = BertClient()

In [ ]:
from nltk.tokenize import word_tokenize
from numpy import savetxt
from gensim.models import KeyedVectors
import nltk
#from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
from nltk.stem.porter import PorterStemmer
import nltk
import pandas as pd
import numpy as np
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
from sklearn import model_selection
from sklearn.utils import class_weight
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn import svm
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score


# If not previously performed:
# nltk.download('stopwords')

stemming = PorterStemmer()
stops = set(stopwords.words("english"))
def apply_cleaning_function_to_list(X):
    cleaned_X = []
    for element in X:
        cleaned_X.append(clean_text(element))
    return cleaned_X


def clean_text(raw_text):
    """This function works on a raw text string, and:
        1) changes to lower case
        2) tokenizes (breaks down into words
        3) removes punctuation and non-word text
        4) finds word stems
        5) removes stop words
        6) rejoins meaningful stem words"""
    
    # Convert to lower case
    text = raw_text.lower()
    
    # Tokenize
    tokens = nltk.word_tokenize(text)
    
    # Keep only words (removes punctuation + numbers)
    # use .isalnum to keep also numbers
    token_words = [w for w in tokens if w.isalpha()]
    
    # Stemming
    stemmed_words = [stemming.stem(w) for w in token_words]
    
    # Remove stop words
    meaningful_words = [w for w in stemmed_words if not w in stops]
    
    # Rejoin meaningful stemmed words
    joined_words = ( " ".join(meaningful_words))
    
    # Return cleaned data
    return joined_words


datas=pd.read_csv('/content/simone2.csv')
dataf=list(datas['text'])
y=list(datas['Label'])
y = np.array(y).astype(int)
#print(dataf[420])

# Get text to clean
text_to_clean = dataf
# Clean text
cleaned_text = apply_cleaning_function_to_list(text_to_clean)


text=cleaned_text

X=[]
for j in range(0,len(text)):
     #print(j)
     tokens = word_tokenize(text[j])
     words = [word for word in tokens if word.isalpha()]

     

     
     sum=0
     for i in range(0,len(words)):
        try:
           sum=sum+bc.encode([words[i]])
      
        except:
           k=0
     X.append(sum)


X = np.array(X).astype(float)
X=np.reshape(X,(len(X),768))
savetxt('BERT-simone.csv', X, delimiter=',')
print(X.shape)
kf=KFold(n_splits=10, random_state=None, shuffle=True)

acc=[]
prec=[]
reca=[]
f=[]

for train_index, test_index in kf.split(X):
        
        #print("TRAIN:", train_index, "TEST:", test_index)
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        #print(y_train)
        clf= LogisticRegression(C = 100,random_state = 0)
        clf.fit(X_train, y_train)
        y_pred = clf.predict( X_test)
        acc.append(accuracy_score(y_test,  y_pred))
        reca.append(recall_score(y_test,  y_pred))
        prec.append(precision_score(y_test,  y_pred))
        f.append(f1_score(y_test,  y_pred))
        
        
print("accuracy : "+ str(np.mean(acc)))
print("recall : "+ str(np.mean(reca)))
print("precision : "+ str(np.mean(prec)))
print("f1 score : " + str(np.mean(f)))






[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
(120, 768)


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


accuracy : 0.825
recall : 0.9262193362193363
precision : 0.8612121212121211
f1 score : 0.885641554867561


In [ ]:
datas=pd.read_csv('/content/tahani2.csv')
dataf=list(datas['text'])
y=list(datas['Label'])
y = np.array(y).astype(int)
#print(dataf[420])

# Get text to clean
text_to_clean = dataf
# Clean text
cleaned_text = apply_cleaning_function_to_list(text_to_clean)


text=cleaned_text

X=[]
for j in range(0,len(text)):
     #print(j)
     tokens = word_tokenize(text[j])
     words = [word for word in tokens if word.isalpha()]

     

     
     sum=0
     for i in range(0,len(words)):
        try:
           sum=sum+bc.encode([words[i]])
      
        except:
           k=0
     X.append(sum)


X = np.array(X).astype(float)
X=np.reshape(X,(len(X),768))
savetxt('BERT-tahani.csv', X, delimiter=',')
print(X.shape)
kf=KFold(n_splits=10, random_state=None, shuffle=True)

acc=[]
prec=[]
reca=[]
f=[]

for train_index, test_index in kf.split(X):
        
        #print("TRAIN:", train_index, "TEST:", test_index)
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        #print(y_train)
        clf= LogisticRegression(C = 100,random_state = 0)
        clf.fit(X_train, y_train)
        y_pred = clf.predict( X_test)
        acc.append(accuracy_score(y_test,  y_pred))
        reca.append(recall_score(y_test,  y_pred))
        prec.append(precision_score(y_test,  y_pred))
        f.append(f1_score(y_test,  y_pred))
        
        
print("accuracy : "+ str(np.mean(acc)))
print("recall : "+ str(np.mean(reca)))
print("precision : "+ str(np.mean(prec)))
print("f1 score : " + str(np.mean(f)))


(126, 768)


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


accuracy : 0.6891025641025641
recall : 0.5526984126984126
precision : 0.5747619047619048
f1 score : 0.5112301587301588


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


In [ ]:
datas=pd.read_csv('/content/laura2.csv')
dataf=list(datas['text'])
y=list(datas['Label'])
y = np.array(y).astype(int)
#print(dataf[420])

# Get text to clean
text_to_clean = dataf
# Clean text
cleaned_text = apply_cleaning_function_to_list(text_to_clean)


text=cleaned_text

X=[]
for j in range(0,len(text)):
     #print(j)
     tokens = word_tokenize(text[j])
     words = [word for word in tokens if word.isalpha()]

     

     
     sum=0
     for i in range(0,len(words)):
        try:
           sum=sum+bc.encode([words[i]])
      
        except:
           k=0
     X.append(sum)


X = np.array(X).astype(float)
X=np.reshape(X,(len(X),768))
savetxt('BERT-laura.csv', X, delimiter=',')
print(X.shape)
kf=KFold(n_splits=10, random_state=None, shuffle=True)

acc=[]
prec=[]
reca=[]
f=[]

for train_index, test_index in kf.split(X):
        
        #print("TRAIN:", train_index, "TEST:", test_index)
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        #print(y_train)
        clf= LogisticRegression(C = 100,random_state = 0)
        clf.fit(X_train, y_train)
        y_pred = clf.predict( X_test)
        acc.append(accuracy_score(y_test,  y_pred))
        reca.append(recall_score(y_test,  y_pred))
        prec.append(precision_score(y_test,  y_pred))
        f.append(f1_score(y_test,  y_pred))
        
        
print("accuracy : "+ str(np.mean(acc)))
print("recall : "+ str(np.mean(reca)))
print("precision : "+ str(np.mean(prec)))
print("f1 score : " + str(np.mean(f)))


(40, 768)


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


accuracy : 0.675
recall : 0.7
precision : 0.6166666666666666
f1 score : 0.6428571428571428


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [ ]:
datas=pd.read_csv('/content/total-sport2.csv')
dataf=list(datas['text'])
y=list(datas['Label'])
y = np.array(y).astype(int)
#print(dataf[420])

# Get text to clean
text_to_clean = dataf
# Clean text
cleaned_text = apply_cleaning_function_to_list(text_to_clean)


text=cleaned_text

X=[]
for j in range(0,len(text)):
     #print(j)
     tokens = word_tokenize(text[j])
     words = [word for word in tokens if word.isalpha()]

     

     
     sum=0
     for i in range(0,len(words)):
        try:
           sum=sum+bc.encode([words[i]])
      
        except:
           k=0
     X.append(sum)


X = np.array(X).astype(float)
X=np.reshape(X,(len(X),768))
savetxt('BERT-total-sport.csv', X, delimiter=',')
print(X.shape)
kf=KFold(n_splits=10, random_state=None, shuffle=True)

acc=[]
prec=[]
reca=[]
f=[]

for train_index, test_index in kf.split(X):
        
        #print("TRAIN:", train_index, "TEST:", test_index)
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        #print(y_train)
        clf= LogisticRegression(C = 100,random_state = 0)
        clf.fit(X_train, y_train)
        y_pred = clf.predict( X_test)
        acc.append(accuracy_score(y_test,  y_pred))
        reca.append(recall_score(y_test,  y_pred))
        prec.append(precision_score(y_test,  y_pred))
        f.append(f1_score(y_test,  y_pred))
        
        
print("accuracy : "+ str(np.mean(acc)))
print("recall : "+ str(np.mean(reca)))
print("precision : "+ str(np.mean(prec)))
print("f1 score : " + str(np.mean(f)))


(286, 768)


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


accuracy : 0.6786945812807883
recall : 0.7265570295717355
precision : 0.6989507469654528
f1 score : 0.7023790303231776


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
